# Conversão

NetCDF para CSV

* Requer `CO2_1deg_month_1850-2013.nc` em `.\data`
* Arquvio de saída em `.\out\co2_data.csv`

# Bibliotecas

In [1]:
import xarray as xr

# Leitura & Análise Básica

> *NetCDF (Network Common Data Form) is a file format for storing multidimensional scientific data (variables) such as temperature, humidity, pressure, wind speed, and direction. Each of these variables can be displayed through a dimension (such as time) by making a layer or table view from the netCDF file.*

* **NetCDF:** uma estrutura conveniente para armazenar dados que possuem dependência espacial (geográfica) e temporal;
* **CSV:** o usual, por isso, ao invés de permanecer noutro formato, é feita a conversão para CSV.

As imagens abaixo dão uma ideia melhor do que se trata o NetCDF:

<img src="https://docs.geoserver.geo-solutions.it/edu/en/_images/md1.png" height="350px" alt="netcdf-exemple-1" />

<img src="https://simulatingcomplexity.files.wordpress.com/2014/11/netcdf-file-structure.png" height="350px" alt="netcdf-exemple-2" />

***

As convenções e padrões encontram-se documentados em [NetCDF Climate and Forecast (CF) Metadata Conventions](https://cfconventions.org/cf-conventions/cf-conventions.html).

In [2]:
nc = xr.open_dataset('./data/CO2_1deg_month_1850-2013.nc')

In [3]:
nc.head()

<xarray.Dataset>
Dimensions:    (LonDim: 5, LatDim: 5, Times: 5)
Coordinates:
  * Times      (Times) datetime64[ns] 1850-01-01 1850-02-01 ... 1850-05-01
Dimensions without coordinates: LonDim, LatDim
Data variables:
    Longitude  (LonDim) float64 ...
    Latitude   (LatDim) float64 ...
    value      (Times, LatDim, LonDim) float64 ...

Sobre o *dataset*:

* É construído apenas valores incrementais (referência + avanço/atraso), com excessão das concetração de $CO_2$, isto é:
    * `Times`
        * É o único índice de fato
        * Segue a lógica "*days since 1850-01-01*" e é portanto `int`
        * Contudo, a biblioteca automaticamente converte para um objeto de data do Python
    * `Longitude`
        * Varre de -179.5 até 179.5 com passo 1
        * Em graus e com referência no Meridiano de Greenwich
    * `Latitude`
        * Varre de -89.5 até 89.5 com passo 1
        * Em graus e com referência na Linha do Equador

<br />

* A presença de .5 faz sentido com o padrão *cell center* estabelecido para leituras em NetCDF

* Finalmente, cada tupla `(Times, Longitude, Latitude)` mapeia para uma (e somente uma) leitura de $CO_2$ (`value`)

* Essas informações podem ser atestadas abaixo:

In [4]:
nc.info()

xarray.Dataset {
dimensions:
	LonDim = 360 ;
	LatDim = 180 ;
	Times = 1968 ;

variables:
	float64 Longitude(LonDim) ;
		Longitude:units = degrees_east ;
		Longitude:Long_name = longitude ;
	float64 Latitude(LatDim) ;
		Latitude:units = degrees_north ;
		Latitude:Long_name = latitude ;
	float64 value(Times, LatDim, LonDim) ;
	datetime64[ns] Times(Times) ;
		Times:Long_name = Times ;
		Times:note = that is the average of the month ;

// global attributes:
}

In [5]:
nc.indexes

Indexes:
    Times    DatetimeIndex(['1850-01-01', '1850-02-01', '1850-03-01', '1850-04-01',
               '1850-05-01', '1850-06-01', '1850-07-01', '1850-08-01',
               '1850-09-01', '1850-10-01',
               ...
               '2013-03-01', '2013-04-01', '2013-05-01', '2013-06-01',
               '2013-07-01', '2013-08-01', '2013-09-01', '2013-10-01',
               '2013-11-01', '2013-12-01'],
              dtype='datetime64[ns]', name='Times', length=1968, freq=None)

In [6]:
nc.data_vars

Data variables:
    Longitude  (LonDim) float64 ...
    Latitude   (LatDim) float64 ...
    value      (Times, LatDim, LonDim) float64 ...

In [7]:
nc.min()

<xarray.Dataset>
Dimensions:    ()
Data variables:
    Longitude  float64 -179.5
    Latitude   float64 -89.5
    value      float64 273.4

In [8]:
nc.max()

<xarray.Dataset>
Dimensions:    ()
Data variables:
    Longitude  float64 179.5
    Latitude   float64 89.5
    value      float64 413.0

# NetCDF ⟶ Pandas ⟶ CSV

1. Parte-se do NetCDF;
2. Chega-se a um `DataFrame` do pandas plano (bidimensional), mas `MultIndex`;
    * Isso pois, como dito, uma tupla mapeia para um valor.
3. Escreve-se o DataFrame em CSV.

In [9]:
df = nc.to_dataframe()
df

Longitude  Latitude       value
LonDim LatDim Times                                      
0      0      1850-01-01     -179.5      89.5  288.134064
              1850-02-01     -179.5      89.5  288.521057
              1850-03-01     -179.5      89.5  288.888397
              1850-04-01     -179.5      89.5  289.094849
              1850-05-01     -179.5      89.5  289.040405
...                             ...       ...         ...
359    179    2013-08-01      179.5     -89.5  393.965240
              2013-09-01      179.5     -89.5  394.114105
              2013-10-01      179.5     -89.5  394.123016
              2013-11-01      179.5     -89.5  394.005280
              2013-12-01      179.5     -89.5  393.901367

[127526400 rows x 3 columns]

A escrita dos dados a seguir leva cerca de 10 minutos e o resultado possui, aproximadamente, 6 GB.

Descomentar e executar se necessário.

In [10]:
# df.to_csv('./out/co2_data.csv', header=True)